In [1]:
import numpy as np


import keras
from keras import models
from keras import layers

import tensorflow as tf

Using TensorFlow backend.


In [2]:
X = np.random.normal(0,1,(1000,50))
import numpy.linalg as lin

# the following is to make X more skewed towards a low rank matrix, so the results
# are more instructive. 
u,l,v = lin.svd(X)
wt = [1.1**(-x) for x in range(len(l))]
l = l * wt
print(l)

X = u @ np.vstack((np.diag(l),np.zeros((950,50)))) @ v.T

# m is the number of coordinates of the input examples, 10 is the 
# number of latent coordinates we will use to reconstruct the data
m = 50
k = 10

[38.1549096  34.16914374 30.51843851 27.54001439 24.92521973 22.5803388
 20.31556874 18.25926376 16.55469272 15.00953804 13.4580718  12.13379529
 10.98172736  9.97753199  8.979406    8.1138124   7.29755663  6.56452899
  5.8796556   5.33594489  4.81722402  4.36209592  3.94931645  3.55532657
  3.19966893  2.89028877  2.61533515  2.36179739  2.14271402  1.93660099
  1.74294661  1.58034335  1.41908014  1.27270045  1.15077674  1.0380125
  0.9380639   0.84339567  0.76028914  0.69031817  0.61881157  0.5601978
  0.49892636  0.44779954  0.40178157  0.35889015  0.32562174  0.29148827
  0.26012688  0.23308897]


In [3]:
autoencoder = models.Sequential()
Wlayer = layers.Dense(k,activation='linear', input_shape=(m,))
autoencoder.add(Wlayer)

Vlayer = layers.Dense(m,activation='linear')
# If you want to fix V and see how W varies, uncomment the following line:
# Vlayer.trainable = False
autoencoder.add(Vlayer)

#Vlayer.set_weights([V.T,np.zeros(50,)])
autoencoder.compile(optimizer='rmsprop', loss='mse')
autoencoder.fit(X,X,epochs=20,batch_size=32)


Epoch 1/20
1000/1000 [==============================] - 0s 322us/step - loss: 0.1923
Epoch 2/20
1000/1000 [==============================] - 0s 103us/step - loss: 0.1460
Epoch 3/20
1000/1000 [==============================] - 0s 393us/step - loss: 0.1206
Epoch 4/20
 256/1000 [======>.......................] - ETA: 0s - loss: 0.1071

C:\Users\prasadsn\Anaconda3\envs\tensorflow_env\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.107438). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


1000/1000 [==============================] - 1s 745us/step - loss: 0.1020
Epoch 5/20
1000/1000 [==============================] - 0s 293us/step - loss: 0.0860
Epoch 6/20
1000/1000 [==============================] - 0s 104us/step - loss: 0.0721
Epoch 7/20
1000/1000 [==============================] - 0s 86us/step - loss: 0.0607
Epoch 8/20
1000/1000 [==============================] - 0s 102us/step - loss: 0.0521
Epoch 9/20
1000/1000 [==============================] - 0s 118us/step - loss: 0.0456
Epoch 10/20
1000/1000 [==============================] - 0s 146us/step - loss: 0.0406
Epoch 11/20
1000/1000 [==============================] - 0s 145us/step - loss: 0.0367
Epoch 12/20
1000/1000 [==============================] - 0s 144us/step - loss: 0.0334
Epoch 13/20
1000/1000 [==============================] - 0s 166us/step - loss: 0.0308
Epoch 14/20
1000/1000 [==============================] - 0s 249us/step - loss: 0.0286
Epoch 15/20
1000/1000 [==============================] - 0s 198us/step -

In [4]:
autoencoder.fit(X,X,epochs=20,batch_size=32)


Epoch 1/20
1000/1000 [==============================] - 0s 179us/step - loss: 0.0216
Epoch 2/20
1000/1000 [==============================] - 0s 106us/step - loss: 0.0213
Epoch 3/20
1000/1000 [==============================] - 0s 108us/step - loss: 0.0210
Epoch 4/20
1000/1000 [==============================] - 0s 98us/step - loss: 0.0208
Epoch 5/20
1000/1000 [==============================] - 0s 160us/step - loss: 0.0206
Epoch 6/20
1000/1000 [==============================] - 0s 48us/step - loss: 0.0204
Epoch 7/20
1000/1000 [==============================] - 0s 159us/step - loss: 0.0203
Epoch 8/20
1000/1000 [==============================] - 0s 89us/step - loss: 0.0202
Epoch 9/20
1000/1000 [==============================] - 0s 218us/step - loss: 0.0202
Epoch 10/20
1000/1000 [==============================] - 0s 99us/step - loss: 0.0201
Epoch 11/20
1000/1000 [==============================] - 1s 752us/step - loss: 0.0200
Epoch 12/20
1000/1000 [==============================] - 1s 656us/s

In [11]:
W = autoencoder.layers[0].get_weights()[0].T
V = autoencoder.layers[1].get_weights()[0].T
print('shape of W: ', W.shape)
print('shape of V: ', V.shape)

shape of W:  (10, 50)
shape of V:  (50, 10)


In [12]:
# Verify that WV =I, the weights W of the first 
# layer is the projection into the column space of the weights of the second layer, V.
# You won't get exactly I because the optimization is stochastic, but close enough

np.round(W @ V,1)

array([[ 1.,  0., -0., -0., -0.,  0.,  0.,  0., -0., -0.],
       [ 0.,  1.,  0.,  0.,  0., -0., -0.,  0., -0., -0.],
       [-0., -0.,  1., -0., -0., -0., -0., -0., -0.,  0.],
       [-0.,  0., -0.,  1.,  0.,  0.,  0., -0.,  0., -0.],
       [ 0.,  0., -0., -0.,  1., -0.,  0.,  0.,  0., -0.],
       [-0., -0., -0.,  0., -0.,  1., -0.,  0., -0., -0.],
       [ 0.,  0., -0., -0., -0.,  0.,  1., -0., -0., -0.],
       [ 0., -0., -0.,  0., -0.,  0.,  0.,  1.,  0., -0.],
       [-0., -0.,  0., -0., -0., -0., -0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  1.]], dtype=float32)

In [13]:
np.round(V @ W, 1)

array([[ 0.2, -0.1, -0.1, ...,  0.1,  0. , -0. ],
       [-0. ,  0.2,  0.1, ...,  0. ,  0. ,  0. ],
       [-0.1,  0.1,  0.1, ..., -0. , -0. ,  0.1],
       ...,
       [ 0. , -0.1, -0. , ...,  0.4,  0. ,  0.1],
       [ 0. ,  0. , -0. , ..., -0.1,  0.1, -0.1],
       [-0. ,  0. ,  0.1, ...,  0.1, -0. ,  0.2]], dtype=float32)

In [6]:
# We build a new model that ties the weights of the W and V layers (ie forces W=V^T) 
# to force the weight matrices V (W resp) of the prior example to have orthogonal 
# columns, ie V^T V = I (rows resp, W W^T = I) 

# This approximates the SVD. To tie the weights together, we will have to 
# extend the keras.layers.Layer class to take in another layer, and use the weights from 
# that layer. The minimal thing we need to do for this is redefine the output computation 
# when the layer is called, and this is achieved by redefining the inbuilt call function of 
# the Layer class. 

class mylayer(layers.Layer):
    def __init__(self, tracklayer=None, activation=None, **kwargs):
        self.kernel_to_track = tracklayer
        self.activation = keras.activations.get(activation)
        super().__init__(**kwargs)
    
    def call(self, inputs):
        output = tf.matmul(inputs, self.kernel_to_track.weights[0], transpose_b = True)
        return self.activation(output)
    


In [7]:
# To define the autoencoder model:

class autoencoder_svd(keras.Model):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        self.encoder = layers.Dense(10, activation = 'linear')
        self.decoder = mylayer(self.encoder, activation = 'linear')
    
    def call(self, inputs):
        w = self.encoder(inputs)
        self.latent = w
        hatx = self.decoder(w)
        return(hatx)


In [8]:
svdnn = autoencoder_svd()

svdnn.compile(optimizer='adam', loss='mse')
svdnn.fit(X,X,epochs=20,batch_size=32)


Epoch 1/20
1000/1000 [==============================] - 0s 152us/step - loss: 0.1432
Epoch 2/20
1000/1000 [==============================] - 0s 112us/step - loss: 0.1224
Epoch 3/20
1000/1000 [==============================] - 0s 63us/step - loss: 0.1052
Epoch 4/20
1000/1000 [==============================] - 1s 669us/step - loss: 0.0872
Epoch 5/20
1000/1000 [==============================] - 0s 157us/step - loss: 0.0716
Epoch 6/20
1000/1000 [==============================] - 0s 44us/step - loss: 0.0594
Epoch 7/20
1000/1000 [==============================] - 0s 184us/step - loss: 0.0503
Epoch 8/20
1000/1000 [==============================] - 0s 101us/step - loss: 0.0442
Epoch 9/20
1000/1000 [==============================] - 0s 81us/step - loss: 0.0401
Epoch 10/20
1000/1000 [==============================] - 0s 116us/step - loss: 0.0371
Epoch 11/20
1000/1000 [==============================] - 1s 623us/step - loss: 0.0347
Epoch 12/20
1000/1000 [==============================] - 0s 169us/

In [9]:
svdnn.fit(X,X,epochs=20,batch_size=32)


Epoch 1/20
1000/1000 [==============================] - 0s 146us/step - loss: 0.0227
Epoch 2/20
1000/1000 [==============================] - 0s 101us/step - loss: 0.0223
Epoch 3/20
1000/1000 [==============================] - 0s 52us/step - loss: 0.0220
Epoch 4/20
1000/1000 [==============================] - 0s 118us/step - loss: 0.0217
Epoch 5/20
1000/1000 [==============================] - 0s 122us/step - loss: 0.0215
Epoch 6/20
1000/1000 [==============================] - 0s 132us/step - loss: 0.0213
Epoch 7/20
1000/1000 [==============================] - 0s 151us/step - loss: 0.0212
Epoch 8/20
1000/1000 [==============================] - 0s 116us/step - loss: 0.0211
Epoch 9/20
1000/1000 [==============================] - 0s 127us/step - loss: 0.0209 0s - loss: 0.02
Epoch 10/20
1000/1000 [==============================] - 0s 193us/step - loss: 0.0209
Epoch 11/20
1000/1000 [==============================] - 0s 124us/step - loss: 0.0208
Epoch 12/20
1000/1000 [=========================

In [10]:
W = svdnn.encoder.weights[0].numpy() 
print(W.shape)

np.round(W.T @W,1)

(50, 10)


array([[ 1., -0., -0.,  0., -0.,  0.,  0., -0., -0.,  0.],
       [-0.,  1., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.],
       [-0., -0.,  1., -0., -0., -0., -0.,  0.,  0.,  0.],
       [ 0.,  0., -0.,  1., -0.,  0., -0.,  0., -0., -0.],
       [-0.,  0., -0., -0.,  1.,  0., -0.,  0., -0.,  0.],
       [ 0.,  0., -0.,  0.,  0.,  1.,  0., -0.,  0., -0.],
       [ 0., -0., -0., -0., -0.,  0.,  1., -0.,  0., -0.],
       [-0., -0.,  0.,  0.,  0., -0., -0.,  1.,  0., -0.],
       [-0.,  0.,  0., -0., -0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0., -0.,  0., -0., -0., -0.,  0.,  1.]], dtype=float32)

In [25]:
# finally, we check how much power the columns of W carry on the top 10 eigen-vectors
# of X^TX, ie the best directions to represent the rows of X.
# The closer these numbers are to 1, the better the column space of W represents the 
# optimal linear space to project the rows of X into. Not all numbers need to be 
# close to 1, but at least some need to be.

V = v[:,0:10]
print(V.shape)


# This is the projection of W into smaller eigen-directions
pwr = lin.inv(V.T @ V) @ V.T @ W

print(np.diag(pwr.T @ pwr))


(50, 10)
[0.95233522 0.99358056 0.95987988 0.9969597  0.9835332  0.99383293
 0.99459085 0.96955484 0.98650035 0.99621449]
